# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [26]:
import os
print(os.getenv("PRICE_DATA"))

../05_src/data/prices/


In [27]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [30]:
import os
import sys
from glob import glob

# Write your code below.
# Append custom source directory to sys.path
sys.path.append(os.getenv('SRC_DIR'))

# Import custom logger
from utils.logger import get_logger
_logs = get_logger(__name__)

# Load the PRICE_DATA environment variable
price_data_dir = os.getenv('PRICE_DATA')


    
# Use glob to find all Parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(price_data_dir, '*.parquet'))

if not parquet_files:
    _logs.error(f"No Parquet files found in {price_data_dir}")
    

# Log the number of files found
_logs.info(f"Found {len(parquet_files)} parquet files in {price_data_dir}")

print(parquet_files)


2025-06-06 22:51:34,198, 2841563897.py, 22, ERROR, No Parquet files found in ../05_src/data/prices/
2025-06-06 22:51:34,200, 2841563897.py, 26, INFO, Found 0 parquet files in ../05_src/data/prices/


[]


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [29]:
# Write your code below.
import dask.dataframe as dd

ddf = dd.read_parquet(parquet_files)

# Function to add features per group (per ticker)
def add_features(df):
    df = df.sort_values("Date")  # Make sure it's sorted by time
    df["Close_lag_1"] = df["Close"].shift(1)
    df["Adj_Close_lag_1"] = df["Adj_Close"].shift(1)
    df["returns"] = (df["Close"] / df["Close_lag_1"]) - 1
    df["hi_lo_range"] = df["High"] - df["Low"]
    return df

# Apply the feature function per ticker
dd_feat = ddf.groupby("Ticker").apply(add_features, meta=ddf)

# Optional: trigger computation (if needed for downstream tasks)
# dd_feat = dd_feat.persist()

_logs.info("Feature engineering completed and assigned to dd_feat")

ValueError: empty urlpath sequence

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [23]:
# Write your code below.
# Convert Dask DataFrame to Pandas (compute all data)
pdf = dd_feat.compute()

# Make sure data is sorted per Ticker and Date for proper rolling calculation
pdf = pdf.sort_values(['Ticker', 'Date'])

# Add 10-day moving average of returns for each Ticker
pdf['returns_ma_10'] = pdf.groupby('Ticker')['returns'].transform(lambda x: x.rolling(10).mean())

_logs.info("Converted to Pandas DataFrame and added 10-day moving average of returns.")



NameError: name 'dd_feat' is not defined

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

Not necessarily but for small to medium datasets fitting in memory, pandas conversion is convenient and efficient.

For large datasets that exceed memory, Dask is a better choice for calculating moving averages efficiently and scalably.


(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.